LLM控制面板

In [1]:
from model import LLMPanel, INTERNLM_TEMPLATE, VICUNA_TEMPLATE
from IPython.display import display

model_list = [
        ("gpt2", "/home/cs/yangyuchen/guoyiqiu/my_models/gpt2"),
        ("llama_7b", "/home/cs/yangyuchen/yushengliao/Medical_LLM/llama-7b-hf"),
        ("llama_13b", "/home/cs/yangyuchen/yushengliao/Medical_LLM/llama-13b"),
        ("llama-2-7b-chat", "/home/cs/yangyuchen/yushengliao/Medical_LLM/llama-2-7b-chat-hugging"),
        ("llama-2-13b-chat", "/home/cs/yangyuchen/guoyiqiu/my_models/Llama-2-13b-chat-ms"),
        ("vicuna_7b", "/home/cs/yangyuchen/yushengliao/Medical_LLM/vicuna-7b/"),
        ("internlm-chat-7b", "/home/cs/yangyuchen/yushengliao/Medical_LLM/internlm-chat-7b"),
        ("internlm-chat-20b", "/home/cs/yangyuchen/guoyiqiu/my_models/internlm-chat-20b"),
    ]

panel = LLMPanel(model_list, chat_template=INTERNLM_TEMPLATE)
display(panel)

LLMPanel(children=(HBox(children=(Dropdown(description='Model:', options=(('gpt2', '/home/cs/yangyuchen/guoyiq…

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Everything is ready. Time cost: 24.10s


LAT

In [ ]:
import json
from model import LLMHooker, LLMHookConfig
HONESTY_EXTRACTION_TEMPLATE_EXP = """<|User|>:Pretend you’re an honest person making statements about the world.<eoh>\n<|Bot|>:{stimulus}"""
HONESTY_EXTRACTION_TEMPLATE_REF = """<|User|>:Pretend you’re a dishonest person making statements about the world.<eoh>\n<|Bot|>:{stimulus}"""
tf_dataset = json.load(open("data/true_false_dataset.json"))

def pca():
    pass

def make_function_stimuli_dataset(tmp_t, tmp_f, dst):
    stimuli_dst = []
    dst_t = [d for d in dst if d['label']]
    dst_f = [d for d in dst if not d['label']]
    length = min(len(dst_t),len(dst_f))
    dst_t = dst_t[:length]
    dst_f = dst_f[:length]
    for t,f in zip(dst_t,dst_f):
        stimuli_dst.append([dict(input=tmp_t.format(stimulus=t['input']),label=True), dict(input=tmp_f.format(stimulus=f['input']),label=False)])
    return stimuli_dst

def lat(mt, stimuli_dst, k=5):
    hook_configs = [LLMHookConfig(module_name='block', layer=l) for l in mt.n_layer]
    with LLMHooker(mt, hook_configs) as hooker:
        for pair in stimuli_dst:
            for d in pair:
                